Perform imports

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col#, countDistinct, count, when, isnan
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
from pyspark.ml import Pipeline
import numpy as np

#temporary hide error reports
import warnings
warnings.filterwarnings('ignore')

Read data

In [0]:
model_data_df = spark.read.parquet("/FileStore/final_data")

In [0]:
model_data_df.printSchema()

root
-- Id: integer (nullable = true)
-- label: integer (nullable = true)
-- features: vector (nullable = true)

In [0]:
model_data_df.show(5)

+---+-----+--------------------+
 Id|label| features|
+---+-----+--------------------+
496|34900|(141,[4,19,20,32,...|
917|35311|(141,[0,19,20,36,...|
969|37900|(141,[2,16,22,34,...|
534|39300|(141,[0,15,20,37,...|
 31|40000|(141,[6,19,21,32,...|
+---+-----+--------------------+
only showing top 5 rows

Train and test datasets

In [0]:
train_data_df, test_data_df = model_data_df.randomSplit([0.8, 0.2])

print(f"train_data_df count: {train_data_df.count()}\ntest_data_df count: {test_data_df.count()}")

train_data_df count: 1177
test_data_df count: 283

----
## Model selection

An important task in ML is model selection, or using data to find the best model or parameters for a given task. This is also called tuning.

Basically, I need to find the answers for following questions:

**I. Which technique** should I choose (linear regression, decision tree, random forest etc.)?

**II. Which value of hyperparameters** should I choose, e.g. if the decision tree with depth 3 is better than the decision tree with depth 5?

**III. Which metric** should I use for comparison of all models?

MLlib supports model selection using tools such as CrossValidator and TrainValidationSplit. These tools require the following items:

**I. Estimator**: algorithm or Pipeline to tune

**II. Set of ParamMaps**: parameters to choose from, sometimes called a “parameter grid” to search over

**III. Evaluator**: metric to measure how well a fitted Model does on held-out test data

At a high level, these model selection tools work as follows:

- They split the input data into separate training and test datasets.
- For each (training, test) pair, they iterate through the set of ParamMaps:
    - For each ParamMap, they fit the Estimator using those parameters, get the fitted Model, and evaluate the Model’s performance using the Evaluator.
- They select the Model produced by the best-performing set of parameters.

### I. Estimator

**Linear Regression**

In our case we choose LinearRegression as Estimator

In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="lrPrediction")

lrPipeline = Pipeline(stages = [lr])

**Logistic Regression**

The same piece of code for LogisticRegression will look like:

`from pyspark.ml.classification import LogisticRegression`

`blr = LogisticRegression(featuresCol="features", labelCol="binLabel", predictionCol="blrPrediction")`

### II. ParamGRidBuilder

**Linear Regression**

All parametrs for Linear regression could be found at the documentation:

https://spark.apache.org/docs/2.4.4/ml-classification-regression.html#regression

https://spark.apache.org/docs/2.4.0/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression

https://spark.apache.org/docs/2.4.4/ml-classification-regression.html#survival-regression

But mains parametrs for **Linear Regression** function are following:
* **loss**: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
* **elasticNetParam**: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
* **regParam**: regularization parameter (>= 0). (default: 0.0)
* **maxIter**: max number of iterations (>= 0). (default: 100, current: 50)
* **tol**: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)

Define parameter grid for linear regression

In [0]:
from pyspark.ml.tuning import ParamGridBuilder

lrParamGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.0, 0.1, 0.5, 1.0]) \
                .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.5])\
                .addGrid(lr.maxIter, [5, 15, 50, 100])\
                .build()

L(w;x,y):=1/2*(wTx−y)^2 + α(λ∥w∥1)+(1−α)(λ/2∥w∥2^2),α∈[0,1],λ≥0

**Logistic Regression**

Main parameters for **Logistic Regression** are nearly the same as in case of Linear Regression.
There is a couple interesting parameter in advance:
- threshold - threshold in binary classification prediction (range [0, 1]) has impact on predicted value. Default is 0.5
- probabilityCol - conditional probability, 1/(1 + exp(-rawPrediction_k))
- rawPredictionCol - Raw prediction for each possible label

### III. Evaluator

The Evaluator can be a RegressionEvaluator for regression problems, a BinaryClassificationEvaluator for binary data, or a MulticlassClassificationEvaluator for multiclass problems. 

**Linear Regression**
RegressionEvaluator options:
- **rmse** - root mean squared error (default)
- **mse** - mean squared error
- **r2** - r^2 metric
- **mae** - mean absolute error

In our case we are satisfied with the existing options of RegressionEvaluator and choose RMSE.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

lrEvaluator = RegressionEvaluator()\
                .setPredictionCol("lrPrediction")\
                .setMetricName("rmse")

**Logistic Regression**
BinaryClassificationEvaluator options:
- **areaUnderROC**
- **areaUnderPR**


The code for Logistic Regression look like this

`from pyspark.ml.evaluation import BinaryClassificationEvaluator`

`lrEvaluator = BinaryClassificationEvaluator().setPredictionCol("blrPrediction").setMetricName("areaUnderROC")`

The great advantage of Spark is that you are not limited on that prepared evaluation metrics. You can create your own metric, for example Lift on the first decile, and choose the best model on the basis of that custom made metric.

### IV. CrossValidator

As we mention above, CrossValidor needs three following items:

- Estimator
- Set of ParamMaps
- Evaluator

There are two more parameters that could have a significant impact mainly on the computing costs:
- **numFolds** - how many splits into training and testing datasets will be done, i.e. how many models will be trained for each parameters combination

- **parallelism** - how many models will be trained in parallel

So **how many models** will be calculated during cross validation? If we have two hyperametrs, each has three values and numFold is set to 3, in total we have
2 x 3 x 3 = 18. In total 18 models will be trained during cross validation. ism parameter

So finally we **set CrossValidator** ...

In [0]:
from pyspark.ml.tuning import CrossValidator

lrCrossValidator = CrossValidator(estimator=lrPipeline,
                        estimatorParamMaps=lrParamGrid,
                        evaluator=lrEvaluator,
                        numFolds=2,
                        parallelism=3)

... and **train CrossValidator**.

In [0]:
lrCrossValidatorModel = lrCrossValidator.fit(train_data_df)

____

## Result of CrossValidator

Not user friendly CrossValidator output of performance

In [0]:
lrCrossValidatorModel.avgMetrics[0:10]

Out[11]: [33633.9426613954,
 36140.99312824145,
 35970.37037794691,
 35985.91653894349,
 33633.9426613954,
 36140.99312824145,
 35970.37037794691,
 35985.91653894349,
 33633.9426613954,
 36140.99312824145]

Not user friendly parametr grid

In [0]:
lrParamGrid[0:5]

Out[12]: [{Param(parent='LinearRegression_8d67baeb53f6', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='maxIter', doc='max number of iterations (>= 0).'): 5},
 {Param(parent='LinearRegression_8d67baeb53f6', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='maxIter', doc='max number of iterations (>= 0).'): 15},
 {Param(parent='LinearRegression_8d67baeb53f6', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='maxIter', doc='max number of iterations (>= 0).'): 50},
 {Param(parent='LinearRegression_8d67baeb53f6', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='maxIter', doc='max number of iterations (>= 0).'): 100},
 {Param(parent='LinearRegression_8d67baeb53f6', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LinearRegression_8d67baeb53f6', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1,
 Param(parent='LinearRegression_8d67baeb53f6', name='maxIter', doc='max number of iterations (>= 0).'): 5}]

In [0]:
#getting param. names

l1 = len(lrParamGrid[0].keys()) 
paramName = []

for i in range(0,l1):
  paramName.append(list(lrParamGrid[0].keys())[i].name)

print(paramName)

['regParam', 'elasticNetParam', 'maxIter']

In [0]:
#getting param. values

l2 = len(lrParamGrid)
paramValues = []

for i in range(0,l2):
  paramValues.append(lrParamGrid[i].values())

print(paramValues)

[dict_values([0.0, 0.0, 5]), dict_values([0.0, 0.0, 15]), dict_values([0.0, 0.0, 50]), dict_values([0.0, 0.0, 100]), dict_values([0.0, 0.1, 5]), dict_values([0.0, 0.1, 15]), dict_values([0.0, 0.1, 50]), dict_values([0.0, 0.1, 100]), dict_values([0.0, 0.5, 5]), dict_values([0.0, 0.5, 15]), dict_values([0.0, 0.5, 50]), dict_values([0.0, 0.5, 100]), dict_values([0.1, 0.0, 5]), dict_values([0.1, 0.0, 15]), dict_values([0.1, 0.0, 50]), dict_values([0.1, 0.0, 100]), dict_values([0.1, 0.1, 5]), dict_values([0.1, 0.1, 15]), dict_values([0.1, 0.1, 50]), dict_values([0.1, 0.1, 100]), dict_values([0.1, 0.5, 5]), dict_values([0.1, 0.5, 15]), dict_values([0.1, 0.5, 50]), dict_values([0.1, 0.5, 100]), dict_values([0.5, 0.0, 5]), dict_values([0.5, 0.0, 15]), dict_values([0.5, 0.0, 50]), dict_values([0.5, 0.0, 100]), dict_values([0.5, 0.1, 5]), dict_values([0.5, 0.1, 15]), dict_values([0.5, 0.1, 50]), dict_values([0.5, 0.1, 100]), dict_values([0.5, 0.5, 5]), dict_values([0.5, 0.5, 15]), dict_values([0.5, 0.5, 50]), dict_values([0.5, 0.5, 100]), dict_values([1.0, 0.0, 5]), dict_values([1.0, 0.0, 15]), dict_values([1.0, 0.0, 50]), dict_values([1.0, 0.0, 100]), dict_values([1.0, 0.1, 5]), dict_values([1.0, 0.1, 15]), dict_values([1.0, 0.1, 50]), dict_values([1.0, 0.1, 100]), dict_values([1.0, 0.5, 5]), dict_values([1.0, 0.5, 15]), dict_values([1.0, 0.5, 50]), dict_values([1.0, 0.5, 100])]

In [0]:
# all together

allLrModels = pd.DataFrame.from_records(paramValues, columns =paramName )
metricName = lrEvaluator.getMetricName()
allLrModels.loc[:,metricName] = lrCrossValidatorModel.avgMetrics

display(allLrModels.sort_values(allLrModels.columns[-1], ascending = True)[0:100])

regParam,elasticNetParam,maxIter,rmse
1.0,0.5,5,32341.337022360385
0.5,0.5,5,32423.097309477584
1.0,0.1,5,32476.35953677753
0.1,0.1,5,32742.327134400417
0.1,0.5,5,32742.32891654085
0.5,0.1,5,32742.33422770695
0.0,0.5,5,33633.9426613954
0.0,0.0,5,33633.9426613954
0.0,0.1,5,33633.9426613954
0.1,0.1,15,35605.30039637413


### The best model

In [0]:
bestLr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="lrPrediction", 
                                    regParam = 1.0, elasticNetParam = 0.1, maxIter = 5)

bestLrModel = bestLr.fit(train_data_df)

By method **bestModel** we can find the best model calculating by crossValidator according to chosen metric (here RMSE)

`bestLrModel = lrCrossValidatorModel.bestModel`

In [0]:
lrPrediction = bestLrModel.transform(test_data_df)
print("Predictions:")
display(lrPrediction["lrPrediction", "label"].toPandas()[0:5])

lrPrediction,label
213072.17325742688,140000
216194.40698051156,200000
114942.93084217823,144000
340375.3720041543,325300
129026.88467595499,139400


In [0]:
intercept = bestLrModel.intercept
print(f"Intercept:\n{intercept}")

Intercept:
-328926.5031985266

In [0]:
coef = bestLrModel.coefficients.toArray
print(f"Coefficients:\n{coef}")

Coefficients:
<bound method DenseVector.toArray of DenseVector([1405.3117, 5256.8294, -1268.2216, -2508.4113, -8436.2594, -8181.6666, 1793.3454, 0.0, -5165.2206, -3286.6918, -2891.7437, 2252.2776, -2934.1541, -2907.7577, 7496.1513, 3864.1917, -4224.7755, 641.3934, -1337.8551, -15707.7489, -1758.4824, 2322.0218, -1227.0786, 0.0, -2633.426, -2864.5133, -1725.5562, 12172.2727, -6312.3763, -3445.9727, -0.0, 12316.2509, -7863.3619, 30938.7155, -9403.3921, 50854.7833, -12782.9971, -6929.7414, -0.0, 0.0, 2047.0788, 0.0, -7115.8179, -10880.0934, 5054.7583, -10983.5798, -6929.7414, -7084.2274, 3342.2783, 27578.4671, -6186.6152, 3807.209, -3135.2836, -1944.1752, -4253.1914, 1461.3305, 4786.1032, -4894.3601, -0.0, 23528.0765, -5724.6356, -6511.7029, 1805.7575, 3075.9872, -4596.9021, -5724.6356, -8333.0441, 3780.5384, -2916.594, -1755.386, -3403.8554, -3403.8729, 4394.1016, -4394.1016, -6671.9851, 1957.5923, 23594.9729, -4679.6972, -3727.8621, 1078.8609, -4848.7195, 2757.6359, -6719.9952, 8845.0153, 25422.3789, 19745.5244, -9296.7725, 11907.443, 20073.199, 0.0, -8365.3748, 3528.2869, 16449.1142, -0.0, -1687.2483, -5148.6241, 20029.7722, -3617.1974, 8654.8217, 2517.274, -3617.1974, -3821.9882, 5901.3398, 38541.4477, -3913.7863, -15425.7831, -21257.8409, 75892.2728, 1337.7653, -3201.9063, -1842.751, -4213.9331, -7960.7965, 1671.6944, -3731.5099, 17919.0727, -5019.9551, -0.0, -2669.6851, 0.0, 13058.4599, -5837.0572, 30708.304, 5203.5345, -6319.7228, 9783.9773, -4343.5534, 19755.7441, -4526.2248, -8113.794, -3974.5779, 16681.9501, -4200.9126, -1092.3454, 1.6214, 78.6031, 146.2411, 18.1106, 17.3697, 25.0192, 51.4417])>

In [0]:
#lets put the coefficients into the table
from itertools import chain

attrs = sorted((attr["idx"], attr["name"]) for attr in 
               (chain(*lrPrediction.schema["features"].metadata["ml_attr"]["attrs"].values())))

pairs = [(name, bestLrModel.coefficients[idx]) for idx, name in attrs]

sorted_pairs = sorted(pairs, key = lambda p: abs(p[1]), reverse=True)
variables = [("intercept", intercept)] + sorted_pairs

results = pd.DataFrame(variables, columns = ["predictors", "coefficients"])
display(results)

predictors,coefficients
intercept,-328926.5031985266
PoolQC_index_vec_Ex,75892.27284473793
OverallQual_index_vec_10,50854.78325995202
GarageQual_index_vec_Ex,38541.44771785655
OverallQual_index_vec_9,30938.715450357082
Neighborhood_index_vec_1,30708.304040508654
ExterQual_index_vec_Ex,27578.467127251308
TotRmsAbvGrd_index_vec_10,25422.378911049622
KitchenQual_index_vec_Ex,23594.972905062707
BsmtQual_index_vec_Ex,23528.076490346095
